# packages 

In [1]:
import numpy as np
import pandas as pd
import xarray
import funcs as f
import os

/Users/pdoubraw/anaconda3/lib/python3.6/site-packages/distributed/utils.py:136: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [Errno 51] Network is unreachable
  RuntimeWarning,


In [2]:
from scipy.interpolate import RectBivariateSpline

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
fs=14
plt.rc("font",size=fs)

### parameters

In [4]:
spacings = {"mynn":333, "ysu":333, "sh":333, "vles":333, "LES":25, "LES_raw":25}

In [5]:
colors = { "LES"   :"k",
          "LES_raw":"k",
          "vles"   :"#808B96",
          "mynn"   :"#641E16",
          "ysu"    :"#A93226",
          "sh"     :"#D98880",}

lss =    { "LES"   :"-",
          "LES_raw":"-",
          "vles"   :"-",
          "mynn"   :"-",
          "ysu"    :"-",
          "sh"     :"-",}

lws =    { "LES"   :2,
          "LES_raw":4,
          "vles"   :2,
          "mynn"   :2,
          "ysu"    :2,
          "sh"     :2}

In [6]:
keys_in_order = ['LES_raw', 'LES', 'vles', 'mynn', 'ysu', 'sh']
sims          = ["LES","vles","mynn","ysu","sh"]
datapath      = "/Volumes/Baby-Orange/XPIA-NCAR/rawest/"
outpath       = "/Volumes/Baby-Orange/XPIA-NCAR/processed/"

### io

In [7]:
df_zi       = pd.read_csv("/Volumes/Baby-Orange/XPIA-NCAR/processed/ziL_all_days.csv", header=None, index_col=[0], parse_dates=True)
z_zi_target = 0.5

In [8]:
convective      = pd.read_csv(os.path.join(outpath,"datetimes_ziL_gt_25.csv"),header=None,index_col=[0])
convective_list = np.array([ pd.Timestamp(l[0]) for l in convective.values.tolist() ])
nt              = len(convective_list)
print(nt)

269


### bulk

In [65]:
plot_cartesian_psd = False
plot_polar_psd     = False
plot_azimuthal_psd = False
save               = True

varname = 'w'

year    = 2015
month   = 3
days    = [20, 21, 28, 29, 30] #20
hours   = np.arange(14,24,1)
minutes = np.arange(0,60,10)

for day in days:
    for hour in hours:
        for minute in minutes:

            datetime     = pd.Timestamp(year=year,month=month,day=day,hour=hour,minute=minute)        

            if datetime not in convective_list:
                print ("Skipping {0:%Y-%m-%d_%H-%M}".format(datetime))
                continue
            else:
                print("{0:%Y-%m-%d_%H-%M}".format(datetime))

            zi           = df_zi.loc[datetime].values[0]


            xy_primes, xy_means = f.get_prime_and_means(datapath, datetime, zi)
            xy_primes_fine      = f.make_finer(xy_primes)    

            f1d    = {}
            f2d    = {}
            ps_2d  = {}
            psd_2d = {}

            # power and power density in cartesian grid
            for key in xy_primes_coarse.keys():
                print("................")    
                print("Simulation : {0}".format(key))
                print("Spacing : {0}".format(spacings[key]))

                arrays  = xy_primes_fine
                if arrays is xy_primes_fine:
                    spacing = 25.0 
                elif arrays is xy_primes_coarse:
                    spacing = 333.0
                else:
                    spacing = spacings[key]
        
                f1d[key], f2d[key], ps_2d[key] = f.get_psd_2d(arrays[key], spacing=spacing)

                #         # ATTEMPT AT SOME SCALING...
                #         if ((key.lower()!="les")and(key.lower()!="les_raw")):
                #             print(key)
                #             print("scaling")
                #             ps_2d[key] = ps_2d[key] * 1e5#/ (90/1200.)**2

                df                             = np.unique(np.diff(f1d[key]))[0]    
                psd_2d[key]                    = ps_2d[key]/np.abs(df)    

            if plot_cartesian_psd:
                for key in xy_primes_coarse.keys():
                    print(key)    
                    fig = f.plot_psd_cartesian(f1d[key],psd_2d[key])#,log10=False)
                    fig.suptitle(key)

            # power and power density in polar grid
            x         = {}
            y         = {}
            theta     = {}
            r         = {}
            psd_polar = {}
            ps_polar  = {}

            for key in psd_2d.keys():
                min_radius = 50.0 
                x[key], y[key], theta[key], r[key], psd_polar[key] = f.psd_cartesian_to_polar(f1d[key],psd_2d[key],thetas=np.arange(135,315.1,0.5),min_radius=min_radius) 
                x[key], y[key], theta[key], r[key], ps_polar[key]  = f.psd_cartesian_to_polar(f1d[key],ps_2d[key],thetas=np.arange(135,315.1,0.5),min_radius=min_radius)         

            if plot_polar_psd:
                for key in xy_primes_coarse.keys():
                    print(key)    
                    fig = f.plot_psd_polar(theta[key],r[key],psd_polar[key])#,log10=False)#,vmin=-5,vmax=7)        

            # power and power density averaged azimuthally
            r_1d   = {}    
            psd_1d = {}
            ps_1d  = {}

            for key in psd_2d.keys():
                psd_1d[key]   = np.nanmean(psd_polar[key],1)
                ps_1d[key]    = np.nanmean(ps_polar[key],1)
                r_1d[key]     = np.nanmean(r[key],1)    

            if plot_azimuthal_psd:

                fig = plt.figure(figsize=(6,6))
                ax  = fig.add_subplot(111)
                ax.grid(color='gray', linestyle=':', linewidth=1)

                for key in keys_in_order:
                    ax.loglog(r_1d[key],psd_1d[key],lw=3,label=key.upper(),color=colors[key], linestyle=lss[key], linewidth=lws[key])

                ax.set_xlabel("Wave number $\kappa=\sqrt{\kappa_x^2+\kappa_y^2}$ [1/m]")
                ax.set_ylabel("Azimuthally averaged, two-dimensional\nPower spectrum [m$^2$/s$^2$]")
                ax.set_title("$z/z_i={0}$".format(z_zi_target)+"\n$t=${0:%Y-%m-%d %H:%M} UTC".format(datetime)+"\nvariable: {0}".format(varname))
                plt.legend()

            if save:
                # save everything for this time THIS WONT WORK IF ALL SIMS ARE NOT ON THE SAME GRID, NOT CODED ROBUSTLY ENOUGH
                fout = os.path.join(outpath,"psd_cartesian_{0:%Y-%m-%d_%H-%M}_{1}.npz".format(datetime,varname))
                print("===> {0}".format(fout))
                np.savez_compressed(fout, f_1d=f1d[key], f_2d=f2d[key], mynn=psd_2d['mynn'], ysu=psd_2d['ysu'], vles=psd_2d['vles'], sh=psd_2d['sh'], les=psd_2d['LES'], les_raw=psd_2d['LES_raw'])

                fout = os.path.join(outpath,"ps_cartesian_{0:%Y-%m-%d_%H-%M}_{1}.npz".format(datetime,varname))
                print("===> {0}".format(fout))
                np.savez_compressed(fout, f_1d=f1d[key], f_2d=f2d[key], mynn=ps_2d['mynn'], ysu=ps_2d['ysu'], vles=ps_2d['vles'], sh=ps_2d['sh'], les=ps_2d['LES'], les_raw=ps_2d['LES_raw'])        

                fout = os.path.join(outpath,"psd_polar_{0:%Y-%m-%d_%H-%M}_{1}.npz".format(datetime,varname))
                print("===> {0}".format(fout))   
                np.savez_compressed(fout, radii=r[key], thetas=theta[key], x=x[key], y=y[key], mynn=psd_polar['mynn'], ysu=psd_polar['ysu'], vles=psd_polar['vles'], sh=psd_polar['sh'], les=psd_polar['LES'], les_raw=psd_polar['LES_raw'])

                fout = os.path.join(outpath,"ps_polar_{0:%Y-%m-%d_%H-%M}_{1}.npz".format(datetime,varname))
                print("===> {0}".format(fout))   
                np.savez_compressed(fout, radii=r[key], thetas=theta[key], x=x[key], y=y[key], mynn=ps_polar['mynn'], ysu=ps_polar['ysu'], vles=ps_polar['vles'], sh=ps_polar['sh'], les=ps_polar['LES'], les_raw=ps_polar['LES_raw'])        

                df = pd.DataFrame(index=r_1d[key],columns=r_1d.keys())
                for key in df.columns:
                    df[key] = psd_1d[key].copy()
                df.columns = [a.lower() for a in df.columns]    
                fout = os.path.join(outpath,"psd_azimuthal_mean_{0:%Y-%m-%d_%H-%M}_{1}.npz".format(datetime,varname))
                print("===> {0}".format(fout))               
                df.to_csv(fout)

                df = pd.DataFrame(index=r_1d[key],columns=r_1d.keys())
                for key in df.columns:
                    df[key] = ps_1d[key].copy()
                df.columns = [a.lower() for a in df.columns]    
                fout = os.path.join(outpath,"ps_azimuthal_mean_{0:%Y-%m-%d_%H-%M}_{1}.npz".format(datetime,varname))
                print("===> {0}".format(fout))               
                df.to_csv(fout)        

2015-03-20_14-00
................
Simulation : LES
Spacing : 25
................
Simulation : LES_raw
Spacing : 25
................
Simulation : vles
Spacing : 333
................
Simulation : mynn
Spacing : 333
................
Simulation : ysu
Spacing : 333
................
Simulation : sh
Spacing : 333
===> /Volumes/Baby-Orange/XPIA-NCAR/processed/psd_cartesian_2015-03-20_14-00_w.npz
===> /Volumes/Baby-Orange/XPIA-NCAR/processed/ps_cartesian_2015-03-20_14-00_w.npz
===> /Volumes/Baby-Orange/XPIA-NCAR/processed/psd_polar_2015-03-20_14-00_w.npz
===> /Volumes/Baby-Orange/XPIA-NCAR/processed/ps_polar_2015-03-20_14-00_w.npz
===> /Volumes/Baby-Orange/XPIA-NCAR/processed/psd_azimuthal_mean_2015-03-20_14-00_w.npz
===> /Volumes/Baby-Orange/XPIA-NCAR/processed/ps_azimuthal_mean_2015-03-20_14-00_w.npz
2015-03-20_14-10
................
Simulation : LES
Spacing : 25
................
Simulation : LES_raw
Spacing : 25
................
Simulation : vles
Spacing : 333
................
Simulation : m

/Users/pdoubraw/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:92: RuntimeWarning: Mean of empty slice
/Users/pdoubraw/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:93: RuntimeWarning: Mean of empty slice


===> /Volumes/Baby-Orange/XPIA-NCAR/processed/psd_cartesian_2015-03-20_17-20_w.npz
===> /Volumes/Baby-Orange/XPIA-NCAR/processed/ps_cartesian_2015-03-20_17-20_w.npz
===> /Volumes/Baby-Orange/XPIA-NCAR/processed/psd_polar_2015-03-20_17-20_w.npz
===> /Volumes/Baby-Orange/XPIA-NCAR/processed/ps_polar_2015-03-20_17-20_w.npz
===> /Volumes/Baby-Orange/XPIA-NCAR/processed/psd_azimuthal_mean_2015-03-20_17-20_w.npz
===> /Volumes/Baby-Orange/XPIA-NCAR/processed/ps_azimuthal_mean_2015-03-20_17-20_w.npz
2015-03-20_17-30
................
Simulation : LES
Spacing : 25
................
Simulation : LES_raw
Spacing : 25
................
Simulation : vles
Spacing : 333
................
Simulation : mynn
Spacing : 333
................
Simulation : ysu
Spacing : 333
................
Simulation : sh
Spacing : 333
===> /Volumes/Baby-Orange/XPIA-NCAR/processed/psd_cartesian_2015-03-20_17-30_w.npz
===> /Volumes/Baby-Orange/XPIA-NCAR/processed/ps_cartesian_2015-03-20_17-30_w.npz
===> /Volumes/Baby-Orange/XPIA

# open saved files to make sure they are ok

In [15]:

year     = 2015
month    = 3
day      = 20
hour     = 14
minute   = 0
varname  = 'w'
datetime = pd.Timestamp(year=year,month=month,day=day,hour=hour,minute=minute)
    
# save everything for this time THIS WONT WORK IF ALL SIMS ARE NOT ON THE SAME GRID, NOT ROBUST ENOUGH
fout = os.path.join(outpath,"psd_cartesian_{0:%Y-%m-%d_%H-%M}_{1}.npz".format(datetime,varname))
print("===> {0}".format(fout))
cartesian = np.load(fout)

fout = os.path.join(outpath,"psd_polar_{0:%Y-%m-%d_%H-%M}_{1}.npz".format(datetime,varname))
print("===> {0}".format(fout))   
polar = np.load(fout)

fout = os.path.join(outpath,"psd_azimuthal_mean_{0:%Y-%m-%d_%H-%M}_{1}.npz".format(datetime,varname))
print("===> {0}".format(fout))       
df = pd.read_csv(fout, index_col=[0])

===> /Volumes/Baby-Orange/XPIA-NCAR/processed/psd_cartesian_2015-03-20_14-00_w.npz
===> /Volumes/Baby-Orange/XPIA-NCAR/processed/psd_polar_2015-03-20_14-00_w.npz
===> /Volumes/Baby-Orange/XPIA-NCAR/processed/psd_azimuthal_mean_2015-03-20_14-00_w.npz


### figures (optional)

#### ==> flow field contours

In [21]:
# keys_in_order = ["LES_raw","LES","vles","mynn","ysu","sh"]

# fig = plt.figure(figsize=(6*5,5))
# axs = {}

# for iax,sim in enumerate(arrays.keys()):
#     axs[iax] = fig.add_subplot(1,6,iax+1,aspect="equal")
    
#     p = axs[iax].contourf(arrays[sim], levels=np.arange(-4,4.1,0.2), cmap='magma')
#     variance = np.var(arrays[sim])
#     axs[iax].set_title(sim+"\n $variance={0:.3f}$ m$^2$/s$^2$".format(variance))
#     #plt.colorbar(p)
    
# fig.subplots_adjust(right=0.8)
# cbar_ax = fig.add_axes([0.82, 0.255, 0.02, 0.49])
# clb = fig.colorbar(p, cax=cbar_ax, format=FormatStrFormatter('%3.1f'))
# clb.set_label('$w\prime$ [cm/s]', labelpad=15, y=0.45)
# clb.ax.tick_params(direction='in')     

# junkyard

In [17]:
from scipy.integrate import simps

In [18]:
from scipy import fftpack

In [19]:
key="LES_raw"

In [20]:
for key in arrays.keys():
    
    print(".............")
    print(key)
    
    # apply fft
    fftarray  = fftpack.fft2(arrays[key])
    npts,npts = arrays[key].shape
    fr        = fftpack.fftfreq(npts, d=spacings[key])
    var1      = simps(simps(np.abs(fftarray), fr), fr)
    print("{0:.3f} cm2/s2".format(var1*100))


    fftarray_shifted = fftpack.fftshift(fftarray)
    psd_2d           = np.abs(fftarray_shifted)**2
    fr               = np.sort(fr)
    var2             = simps(simps(np.abs(psd_2d), fr), fr)
    print("{0:.3f} cm2/s2".format(var2*100))
    

.............
LES
-11.438 cm2/s2
459369.340 cm2/s2
.............
LES_raw
1902.697 cm2/s2
520327.772 cm2/s2
.............
vles
0.020 cm2/s2
10.484 cm2/s2
.............
mynn
0.004 cm2/s2
10.128 cm2/s2
.............
ysu
0.012 cm2/s2
8.228 cm2/s2
.............
sh
0.012 cm2/s2
8.227 cm2/s2


In [136]:
for key in keys_in_order:
    print (key, simps(simps(psd_2d[key], f1d[key]), f1d[key]))

LES_raw 18.505855486800726
LES 4.224238849404005
vles 3.0188613995547425e-05
mynn 4.363883293651493e-05
ysu 4.224404909962288e-05
sh 4.2233218071212604e-05
